# HF model_id로 부터 배포

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

HF_TOKEN = os.environ.get("HF_TOKEN")

In [5]:
from sagemaker import Model
import sagemaker, boto3

region = "us-west-2"
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

model_id = "google/gemma-3-4b-it"

# LMI v15 (vLLM) 컨테이너 URI (버전은 예시)
container_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128"


# vLLM Async 모드(권장). LoRA가 필요 없다면 성능/안정성 좋음
env = {
    "HF_MODEL_ID": model_id,
    # "HF_MODEL_ID": "s3://my-model-train/hf_models/gemma-3-4b-it-uncompressed/",
    "OPTION_ASYNC_MODE": "true",
    "OPTION_ROLLING_BATCH": "disable",
    "OPTION_ENTRYPOINT": "djl_python.lmi_vllm.vllm_async_service",
    "OPTION_MAX_MODEL_LEN": "8192",
    "SERVING_FAIL_FAST": "true",
    "HF_TOKEN": HF_TOKEN, # "<your token>"
}

model = Model(image_uri=container_uri, role=role,  env=env)
endpoint_name = sagemaker.utils.name_from_base("gemma3-s3-vllm-async")

print(endpoint_name)

gemma3-s3-vllm-async-2025-08-13-10-48-23-372


In [6]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.24xlarge",
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=500,  # 대형 모델 로딩 여유
)

------------------!

In [7]:
import boto3, json
smr = boto3.client("sagemaker-runtime", region_name=region)

body = {
  "messages":[{"role":"user","content":"가을 캠핑 체크리스트 만들어줘"}],
  "max_tokens":256,
  "temperature":0.7,
  "stream":False
}

resp = smr.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(body),
    ContentType="application/json"
)
print(resp["Body"].read().decode())

{"id":"chatcmpl-d957dd896621418c84db47e192c5a588","object":"chat.completion","created":1755082754,"model":"lmi","choices":[{"index":0,"message":{"role":"assistant","reasoning_content":null,"content":"가을 캠핑 체크리스트를 만들어 드릴게요! 🍂🍁 가을 캠핑은 여름만큼 뜨겁지 않지만, 쌀쌀한 날씨와 변덕스러운 날씨 때문에 꼼꼼한 준비가 필요합니다. \n\n**1. 캠핑 장비 점검 및 준비**\n\n*   [ ] 텐트: 방수, 방풍 기능 확인 및 팩, 스틱 등 부속품 점검\n*   [ ] 타프: 바람에 잘 맞는지, 팩이 잘 박히는지 확인\n*   [ ] 테이블, 의자: 튼튼한지, 수납 가능한지 확인\n*   [ ] 랜턴, 헤드랜턴: 여분 배터리 준비\n*   [ ] 버너, 연료: 작동 확인 및 연료 충분히 준비\n*   [ ] 코펠, 식기, 조리도구: 깨끗하게 세척 후 준비\n*   [ ] 보관함, 수납 가방: 짐 정리 용이하도록 준비\n*   [ ] 캠핑 매트, 침낭: 텐트 온도를 고려하여 침낭 선택\n*   [ ] 난방 용품 (선택): 전기히터, 보","tool_calls":[]},"logprobs":null,"finish_reason":"length","stop_reason":null}],"usage":{"prompt_tokens":18,"total_tokens":274,"completion_tokens":256,"prompt_tokens_details":null},"prompt_logprobs":null}



# S3로 부터 배포

In [1]:
# instance_type = "ml.p4d.24xlarge"
instance_type = "ml.g6.24xlarge"
# instance_type = "ml.g5.24xlarge"

In [2]:
from sagemaker import Model
import sagemaker, boto3

region = "us-west-2"
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

model_id = "google/gemma-3-4b-it"

# LMI v15 (vLLM) 컨테이너 URI (버전은 예시)
container_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128" # Nvidia Driver 550을 요구 (p5계열)
# container_uri = "763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.31.0-lmi13.0.0-cu124-v1.0"



s3_uri = "s3://my-model-train/hf_models/gemma-3-4b-it-uncompressed/"

# # S3 prefix(끝에 /) 아래에 config.json, *.safetensors, tokenizer.* 등이 그대로 있는 상태
# model_data = {
#     "S3DataSource": {
#         "S3Uri": "s3://my-model-train/hf_models/gemma-3-4b-it-uncompressed/",  # 병합 모델이라면 이 경로
#         "S3DataType": "S3Prefix",
#         "CompressionType": "None"
#     }
# }

# vLLM Async 모드(권장). LoRA가 필요 없다면 성능/안정성 좋음
env = {
    "HF_MODEL_ID": s3_uri,
    "OPTION_ASYNC_MODE": "true",
    "OPTION_ROLLING_BATCH": "disable",
    # "OPTION_ROLLING_BATCH": "vllm",
    # "OPTION_MAX_ROLLING_BATCH_SIZE": "8",
    # "OPTION_MODEL_IMPL": "transformers",
    "OPTION_TENSOR_PARALLEL_DEGREE": "max",
    "OPTION_DTYPE": "auto",  #"bfloat16",
    "OPTION_TRUST_REMOTE_CODE": "true",
    "OPTION_ENTRYPOINT": "djl_python.lmi_vllm.vllm_async_service",
    "OPTION_MAX_MODEL_LEN": "4096",
    "SERVING_FAIL_FAST": "true",
    "OPTION_GPU_MEMORY_UTILIZATION": "0.75",
    "OPTION_SWAP_SPACE": "4",
    # 필요 시 커스텀 모델 파일이 있으면:
    # "OPTION_TRUST_REMOTE_CODE": "true",
}

# env = {
#     "HF_MODEL_ID": s3_uri,
#     "OPTION_ROLLING_BATCH": "vllm",
#     "OPTION_DTYPE": "auto",
#     "OPTION_GPU_MEMORY_UTILIZATION": "0.8",      # 0.7에서 0.6으로 감소
#     "OPTION_MAX_ROLLING_BATCH_SIZE": "4",        # 8에서 4로 감소
#     "OPTION_MAX_NUM_BATCHED_TOKENS": "1024",      # 512에서 256으로 감소
#     "OPTION_TENSOR_PARALLEL_DEGREE": "max",        # max 대신 명시적으로 4
#     "OPTION_MAX_MODEL_LEN": "512",              # 512에서 1024로 증가
#     "OPTION_TRUST_REMOTE_CODE": "true",
#     "OPTION_ASYNC_MODE": "true",
#     # "OPTION_PREEMPTION_MODE": "swap",
#     "OPTION_SWAP_SPACE": "16",
#     "OPTION_ENTRYPOINT": "djl_python.lmi_vllm.vllm_async_service",
# }

# model = Model(image_uri=container_uri, role=role, model_data=model_data, env=env)
model = Model(image_uri=container_uri, role=role,  env=env)
endpoint_name_s3 = sagemaker.utils.name_from_base("gemma3-s3-vllm-async")

print(endpoint_name_s3)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
gemma3-s3-vllm-async-2025-08-14-02-10-03-005


In [3]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name_s3,
    container_startup_health_check_timeout=500,  # 대형 모델 로딩 여유
)

------------------!

In [4]:
import boto3, json
smr = boto3.client("sagemaker-runtime", region_name=region)

body = {
  "messages":[{"role":"user","content":"가을 캠핑 체크리스트 만들어줘"}],
  "max_tokens":256,
  "temperature":0.7,
  "stream":False
}

resp = smr.invoke_endpoint(
    EndpointName=endpoint_name_s3,
    Body=json.dumps(body),
    ContentType="application/json"
)
print(resp["Body"].read().decode())

{"id":"chatcmpl-62ee558c6dc94e7f9158a9fa553b4ffe","object":"chat.completion","created":1755137979,"model":"lmi","choices":[{"index":0,"message":{"role":"assistant","reasoning_content":null,"content":"가을 캠핑 체크리스트를 만들어 드릴게요! 가을 캠핑은 여름만큼 뜨겁지 않지만, 밤에는 쌀쌀해지기 때문에 꼼꼼하게 준비해야 합니다. \n\n**1. 캠핑 장비 점검 및 준비**\n\n*   [ ] 텐트: 찢어짐, 마모, 방수 상태 확인 및 방수 스프레이 뿌리기\n*   [ ] 타프: 찢어짐, 마모, 방수 상태 확인\n*   [ ] 테이블, 의자: 파손 여부 확인, 이동 바구니 준비\n*   [ ] 랜턴, 헤드랜턴: 배터리 충전 및 새 배터리 준비\n*   [ ] 버너, 연료: 버너 작동 확인, 연료 충분히 보충\n*   [ ] 코펠, 식기류: 세척 및 준비, 칼, 도마, 조리도구\n*   [ ] 냉장고/쿨러: 아이스박스 아이스팩/얼음 준비\n*   [ ] 침낭, 매트: 청소 및 점검, 보온에 신경 쓰기\n*   [ ] 캠핑 가방: 짐 정리 및 이동","tool_calls":[]},"logprobs":null,"finish_reason":"length","stop_reason":null}],"usage":{"prompt_tokens":18,"total_tokens":274,"completion_tokens":256,"prompt_tokens_details":null},"prompt_logprobs":null}

